**This is the notebook cotaining the code for the AI Crowd 2022 AI Blitz Age Prediction Challenge. We achieved a F1 Score of 0.870 and a Accuracy Score of 0.871 using YOLO.**

install aicrowd client

In [ ]:
!pip install aicrowd-cli
%load_ext aicrowd.magic

install all librarys as well as clone the YOLOv5 github repo

In [ ]:
#Reading the file
import pandas as pd
import numpy as np
import os

# Image Reading & Preprocessing
from PIL import ImageDraw
import PIL.Image
import cv2
from matplotlib import image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

# Misc.
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestClassifier
import copy
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images
import shutil


print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 11142, done.
remote: Total 11142 (delta 0), reused 0 (delta 0), pack-reused 11142
Receiving objects: 100% (11142/11142), 11.14 MiB | 22.36 MiB/s, done.
Resolving deltas: 100% (7699/7699), done.
/content/yolov5
     |████████████████████████████████| 596 kB 4.6 MB/s 
     |████████████████████████████████| 67 kB 2.0 MB/s 
     |████████████████████████████████| 178 kB 43.5 MB/s 
     |████████████████████████████████| 145 kB 44.3 MB/s 
     |████████████████████████████████| 138 kB 38.9 MB/s 
     |████████████████████████████████| 1.1 MB 32.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requ

aicrowd login

In [ ]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/u5lApIrxCgHpZ28o5yxa7605MSDmdaobZ0S3Mq6gFcM
API Key valid
Gitlab access token valid
Saved details successfully!


In [ ]:
%cd /content
!rm -rf /content/datasets 
!mkdir /content/datasets

%cd datasets
!rm -rf age_prediction
!mkdir age_prediction

%cd age_prediction
!rm -rf images
!mkdir images
!rm -rf labels
!mkdir labels

%cd images
!rm -rf train
!mkdir train
!rm -rf val
!mkdir val
!rm -rf test
!mkdir test

%cd ..
%cd labels
!rm -rf train
!mkdir train
!rm -rf val
!mkdir val
!rm -rf test
!mkdir test

/content
/content/datasets
/content/datasets/age_prediction
/content/datasets/age_prediction/images
/content/datasets/age_prediction
/content/datasets/age_prediction/labels


In [ ]:
#download datasets
%cd ..
%cd ..
%cd ..
%aicrowd ds dl -c age-prediction -o datasets

/content/datasets/age_prediction
/content/datasets
/content


sample_submission.csv:   0%|          | 0.00/36.0k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/36.0k [00:00<?, ?B/s]

test.zip:   0%|          | 0.00/176M [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/48.3k [00:00<?, ?B/s]

train.zip:   0%|          | 0.00/236M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

val.zip:   0%|          | 0.00/117M [00:00<?, ?B/s]

In [ ]:
%cd /content/datasets
#unzip data into created folders
#since we want to use the validation set for training as well, we move it directly into the train folder.
#If you don't want to do that, use line 5 instead of 6.
!unzip train.zip -d age_prediction/images/train > /dev/null
# !unzip val.zip -d age_prediction/images/val> /dev/null
!unzip val.zip -d age_prediction/images/train > /dev/null
!unzip test.zip -d age_prediction/images/test > /dev/null

/content/datasets


read the csv files for each the train, validation and test set

In [ ]:
train = pd.read_csv("/content/datasets/train.csv")
val = pd.read_csv("/content/datasets/val.csv")
#since we merged the images, we also need to merge the corresponding data.
train_new = pd.concat([train, val]).reset_index()
train_new = train_new.drop(columns='index')
# we overwrite the old train with the new train data.
train_new.to_csv("/content/datasets/train.csv")

In [ ]:
#load new train data set
df = pd.read_csv("/content/datasets/train.csv")
df = df.drop(columns='Unnamed: 0')

crop images and save into new folder

In [ ]:
path = "/content/datasets/age_prediction/images/train"
!rm -rf "/content/datasets/age_prediction/images/train_croped"
!mkdir "/content/datasets/age_prediction/images/train_croped"
path2 = "/content/datasets/age_prediction/images/train_croped"
failed = []
for img in os.listdir(path):
  img_name = img.replace('.jpg','')
  try:
    with PIL.Image.open(f"{path}/{img}") as im:
      xcenter = im.width/2
      ycenter = im.height/2
      value = 140
      x1 = xcenter - value
      y1 = ycenter - value
      x2 = xcenter + value
      y2 = ycenter + value
      croped_img = im.crop((x1, y1, x2, y2)).resize((512, 512))
      croped_img.save(f"{path2}/{img_name}_croped.jpg")
      new_name = f'{img_name}_croped'
      df = df.append({'ImageID':f'{new_name}', 'age':df.query(f"ImageID == '{img_name}'").age.item()}, ignore_index = True)
  except:
    print(img, "failed!")
    failed.append(img)
    continue

eba6s.jpg failed!


move croped files to right folder

In [ ]:
#move files to right folder
source_dir = "/content/datasets/age_prediction/images/train_croped"
target_dir = "/content/datasets/age_prediction/images/train"  
file_names = os.listdir(source_dir)
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)
print(len(os.listdir("/content/datasets/age_prediction/images/train"))) #should be 5999*2=11998
#since we have failed images, we need to update our train data set
df.to_csv("/content/datasets/train.csv")

11998


the code below generate the labels for the training data such that YOLO can work with it

In [ ]:
def builder(name):

  df = pd.read_csv(f"/content/datasets/{name}.csv")

  for n in range(len(df)):
    X  = 0.5
    Y  = 0.5
    width = 0.8
    height = 0.8
    age_class = -1

    #get age type as int
    if(df.iloc[n].age == "0-10"):
      age_class =  0
    elif(df.iloc[n].age == "10-20"):
      age_class =  1
    elif(df.iloc[n].age == "20-30"):
      age_class =  2
    elif(df.iloc[n].age == "30-40"):
      age_class =  3
    elif(df.iloc[n].age == "40-50"):
      age_class =  4
    elif(df.iloc[n].age == "50-60"):
      age_class =  5
    elif(df.iloc[n].age == "60-70"):
      age_class =  6
    elif(df.iloc[n].age == "70-80"):
      age_class =  7
    elif(df.iloc[n].age == "80-90"):
      age_class =  8
    elif(df.iloc[n].age == "90-100"):
      age_class =  9
    else:
      print("unknown age type!!!")#let's hope this does not happen


    #write txt for YOLO lable
    f= open(f'/content/datasets/age_prediction/labels/{name}/' +  str(df.iloc[n].ImageID) + ".txt","w+")
    YOLO_string = str(age_class) +" "+ str(X) +" "+ str(Y) +" "+ str(width) + "  " + str(height)
    #print("YOLO_string:", YOLO_string)
    f.write( YOLO_string )
    f.close()

In [ ]:
builder("train")

As a final step we now have to creat a new file called "prediction.yml" in the yolov5/data directory:

In [ ]:
%cd /content/yolov5
prediction_string = "train: /content/datasets/prediction/images/train" + '\n'
prediction_string +=  "val: /content/datasets/prediction/images/train" + '\n' #change to /content/datasets/prediction/images/val if using val set
prediction_string +=  "nc: 10" + '\n'
prediction_string +=  "names: [‘0-10’, ‘10-20’,‘20-30’, ‘30-40’,‘40-50’,‘50-60’,‘60-70’, ‘70-80’, ‘80-90’,‘90-100’]" + '\n'

file1 = open("/content/yolov5/data/prediction.yml","w")
# L = ["This is Delhi \n","This is Paris \n","This is London \n"] 
file1.writelines(prediction_string)
file1.close()


/content/yolov5


In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.0 MB/s 
     |████████████████████████████████| 144 kB 53.0 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

now we can train the model

In [ ]:
#hyperparamters for training

In [ ]:
#hyperparamters for training can be change fluently. 
!python train.py --img 512 --batch 16 --epochs 200 --data ./data/prediction.yml --weights /weights/yolov5l.pt


after training we want to predict on the test set. You have to change the path where the results where logged. The train output will tell you what your path is. It will be called "runs/train/expX", where X will be a number. 

In [ ]:
#CHANGE PATH
!python detect.py --source /content/datasets/age_prediction/images/test/ --weights /content/yolov5/runs/train/"""ADD HERE, e.g. exp1"""/weights/best.pt --save-txt
#!python detect.py --source /content/datasets/age_prediction/images/test/ --weights /content/yolov5/runs/train/exp1/weights/best.pt --save-txt

we now generate the sumbission file

In [ ]:


path_to_found_labels = "/content/yolov5/runs/detect/exp5/labels/" # !!! adjust this for your file-structure  !!!

age_types = ["0-10","10-20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]

sample_submission = pd.read_csv("/content/datasets/sample_submission.csv")
submission = 'ImageID,age\n'

for i in range(len(sample_submission)):
  image_name = sample_submission.iloc[i].ImageID
  
  label_path = path_to_found_labels + image_name + ".txt"

  try:
    f = open(  label_path, "r")
    values = f.read()
    print(image_name,"  ", values)
    values_indiv = list(map(float, values.split()))
    print(values_indiv[0])
    f.close()
  except:
    label_path = path_to_found_labels + "smvsq" + ".txt"
    f = open(  label_path, "r")
    values = f.read()
    print(image_name,"  ", values)
    values_indiv = list(map(float, values.split()))
    print(values_indiv[0])
    f.close()

  agetype = age_types[int(values_indiv[0])]
  
  submission = submission + image_name + ","  +agetype + '\n'

f = open("/content/submission.csv", "a")
f.write(submission)
f.close()
!zip -r /content/submission.zip /content/submission/
#You can download the submission file visible in your directory.